## Libs

In [179]:
from binance.client import Client
from binance.enums import HistoricalKlinesType
from datetime import datetime, timedelta
import pandas as pd
import pandas_ta as ta
pd.set_option('display.max_rows', 500)
import requests,json
import plotly.graph_objs as go

In [180]:
# Define column names for the DataFrame
colnames = [
    'time', 'Open', 'High', 'Low', 'Close', 'vl_volume', 'Close time',
    'Quote asset vl_volume', 'Number of trades', 'Taker buy base asset vl_volume',
    'Taker buy quote asset vl_volume', 'Ignore'
]

# Get all USDT pairs
BASE_URL = 'https://api.kucoin.com'
resp = requests.get(BASE_URL+'/api/v2/symbols')
ticker_list = json.loads(resp.content)
symbols = [ticker['symbol'] for ticker in ticker_list['data'] if str(ticker['symbol'][-4:]) == 'USDT']


interval = '1d'  # Supports intervals like 1m, 1H, 1D, etc.
start = (datetime.today() - timedelta(days=90)).strftime('%m/%d/%Y 00:00:00')   # Start date
end = datetime.today().strftime('%m/%d/%Y 00:00:00') # End date


In [181]:
dicionario_vl_adx = {
    '0-25': '1. Absent or Weak Trend',
    '25-50': '2. Strong Trend',
    '50-75': '3. Very Strong Trend',
    '75-100': '4. Extremely Strong Trend'
}

def check_adx(value):
    for key, name in dicionario_vl_adx.items():
        range_start, range_end = map(int, key.split('-'))
        if range_start <= value <= range_end:
            return name
    return None

# Function to count positive values and reset count on encountering negative value
def count_positive_reset(df_column):
    count = 0
    counts = []

    for value in df_column:
        if value > 0:
            count += 1
        else:
            count = 0  # Reset count if a negative value is encountered
        counts.append(count)

    return counts

In [182]:
def getting_data(symbol):
    kucoin = requests.get(f'https://api.kucoin.com/api/v1/market/candles?type=1day&symbol={symbol}')
    if kucoin.status_code == 200:  # if result is GOOD
        data = json.loads(kucoin.text)  # load the json response from API
        df = pd.DataFrame(data['data'], columns=['unix', 'price_open', 'price_close', 'price_high', 'price_low', 'volume', 'turnover'])  # create dataframe out of json response
        df['nm_symbol'] = f'{symbol.replace("-", "")}'
        df['dt_date'] = pd.to_datetime(df['unix'].astype(int), unit='s')
        df.sort_values(by='dt_date', inplace = True) 
        
    # Add a human readable date
    else:
        print('Error on connection')
        pass

    df[['price_open', 'price_close', 'price_high', 'price_low']] = df[['price_open', 'price_close', 'price_high', 'price_low']].map(lambda x: float(x))

    # Calculate ADX, DI+ and DI- manually
    df[['vl_adx', 'vl_dmp', 'vl_dmn']] = ta.adx(df['price_high'], df['price_low'], df['price_close'], length = 14)
    df['vl_adx_trend'] = df['vl_adx'].apply(check_adx)



    # Calculate ichimoku cloud  
    df[['vl_leading_span_a', 'vl_leading_span_b', 'vl_conversion_line', 'vl_base_line', 'vl_lagging_span']] = ta.ichimoku(df['price_high'], df['price_low'], df['price_close'])[0]
    df['vl_price_over_conv_line'] = df['price_close'] - df['vl_conversion_line']
    df['qt_days_ichimoku_positive'] = count_positive_reset(df['vl_price_over_conv_line'])

    # MACD
    df[['vl_macd', 'vl_macd_hist', 'vl_macd_signal']] = ta.macd(df['price_close']).map(lambda x: float(x))
    df['vl_macd_delta'] = df['vl_macd'] - df['vl_macd_signal']
    df['qt_days_macd_delta_positive'] = count_positive_reset(df['vl_macd_delta'])

    # # Supertrend
    df[['vl_supertrend_trend', 'vl_supertrend_direction', 'vl_supertrend_long', 'vl_supertrend_short']] = ta.supertrend(df['price_high'], df['price_low'], df['price_close'])
    df['qt_days_supertrend_positive'] = count_positive_reset(df['vl_supertrend_direction'])

    df.rename(columns={
        'price_open':'vl_price_open', 
        'price_close':'vl_price_close', 
        'price_high':'vl_price_high', 
        'price_low':'vl_price_low', 
        'volume':'vl_volume'
    }, inplace=True)

    return df


In [183]:
testing = []

total_items = len(symbols)
for index, crypto in enumerate(symbols[:50], start=1):
    try:
        data = getting_data(crypto)
        if not data.empty:  # Check if the dataframe is not empty before appending
            testing.append(data)
            print(f"Item {index}/{total_items}: {crypto}")
    except:
        pass

if testing:  # Check if the list is not empty before concatenating
    df_concat = pd.concat(testing, ignore_index=True)
else:
    # Handle the case when no data is available
    df_concat = pd.DataFrame()

Item 1/801: AVA-USDT
Item 2/801: MTV-USDT
Item 3/801: KMD-USDT
Item 4/801: RFOX-USDT
Item 5/801: TEL-USDT
Item 6/801: TT-USDT
Item 7/801: AERGO-USDT
Item 8/801: XMR-USDT
Item 9/801: ATOM-USDT
Item 10/801: FTM-USDT
Item 11/801: VSYS-USDT
Item 12/801: CHR-USDT
Item 13/801: COTI-USDT
Item 14/801: BNB-USDT
Item 15/801: ALGO-USDT
Item 16/801: XEM-USDT
Item 17/801: XTZ-USDT
Item 18/801: ZEC-USDT
Item 19/801: ADA-USDT
Item 20/801: BOLT-USDT
Item 21/801: ARPA-USDT
Item 22/801: CHZ-USDT
Item 23/801: NOIA-USDT
Item 24/801: WIN-USDT
Item 25/801: BTT-USDT
Item 26/801: EOSC-USDT
Item 27/801: ONE-USDT
Item 28/801: TOKO-USDT
Item 29/801: LUNA-USDT
Item 30/801: SXP-USDT
Item 31/801: AKRO-USDT
Item 32/801: MAP-USDT
Item 33/801: AMPL-USDT
Item 34/801: DAG-USDT
Item 35/801: KPOL-USDT
Item 36/801: ARX-USDT
Item 37/801: NWC-USDT
Item 38/801: BEPRO-USDT
Item 39/801: VRA-USDT
Item 40/801: KSM-USDT
Item 41/801: DASH-USDT
Item 42/801: SUTER-USDT
Item 43/801: SENSO-USDT
Item 44/801: XDB-USDT
Item 45/801: SYLO-U

In [192]:
today = datetime.today().date()
df_indicators = df_concat.loc[
                (df_concat['vl_adx'] > 25) & 
                (df_concat['dt_date'].dt.date == today) &

                # Ichimoku com menos preço maior do que linha de conversão e linha de base
                (df_concat['vl_price_low'] > df_concat['vl_conversion_line']) &
                (df_concat['vl_price_low'] > df_concat['vl_base_line']) &

                # vl_macd histograma maior do que 0 e signal maior do que vl_macd
                (df_concat['vl_macd_hist'] > 0) &
                (df_concat['vl_macd_delta'] > 0.01) &
                (df_concat['qt_days_supertrend_positive'] > 1)
                ]

## Possibilities of Profit

In [195]:
df_indicators[
    [
        'nm_symbol', 
        'dt_date', 
        'vl_price_open', 
        'vl_volume', 
        'vl_adx', 
        'vl_adx_trend',
        'qt_days_ichimoku_positive', 
        'vl_macd_delta',
        'qt_days_macd_delta_positive', 
        'qt_days_supertrend_positive',
        
        ]
        ].sort_values(by=['vl_adx_trend', 'vl_macd_delta'], ascending=False).reset_index()

,index,nm_symbol,dt_date,vl_price_open,vl_volume,vl_adx,vl_adx_trend,qt_days_ichimoku_positive,vl_macd_delta,qt_days_macd_delta_positive,qt_days_supertrend_positive
0,3999,KSMUSDT,2023-12-07,26.67700,332.7382,30.691469,2. Strong Trend,8,0.075203,4,46
1,4699,STXUSDT,2023-12-07,1.03350,1081928.2891,34.019394,2. Strong Trend,15,0.044372,12,100
2,3299,AMPLUSDT,2023-12-07,1.54468,46001.43,33.363201,2. Strong Trend,5,0.039296,4,4
3,2899,LUNAUSDT,2023-12-07,1.03720,837311.47231002,49.030135,2. Strong Trend,7,0.032314,11,41


## Follow up of orders

In [186]:
order_book = pd.read_json('order_book.json').T.reset_index()
order_book['index'] = order_book['index'].replace('-', '', regex=True)
open_orders = order_book['index'].loc[~order_book['sold_date'].notnull()]

In [187]:
df_fup = df_concat.loc[
                (df_concat['nm_symbol'].isin(open_orders) ) & 
                (df_concat['dt_date'].dt.date == today)
                ]

In [188]:
df_fup[['nm_symbol', 'dt_date', 'vl_price_open', 'vl_volume', 'vl_adx', 'vl_adx_trend','qt_days_ichimoku_positive', 'vl_macd_delta','qt_days_macd_delta_positive', 'vl_supertrend_direction']].sort_values(by=['vl_adx'], ascending=False).reset_index()

,index,nm_symbol,dt_date,vl_price_open,vl_volume,vl_adx,vl_adx_trend,qt_days_ichimoku_positive,vl_macd_delta,qt_days_macd_delta_positive,vl_supertrend_direction
0,899,ATOMUSDT,2023-12-07,9.7446,36889.7119,30.384316,2. Strong Trend,13,0.001479,3,1


## Charts

In [189]:
def vl_adx_chart(symbol): 
    df = df_concat.loc[df_concat['nm_symbol'] == f'{symbol}']
    
    # Create the vl_adx trace
    vl_adx_line = go.Scatter(
        x=df['dt_date'],
        y=df['vl_adx'],
        mode='lines',
        name='vl_adx',
        line=dict(color='black')  # Color for vl_adx line
    )

    # Create the DI+ trace
    plus_di_line = go.Scatter(
        x=df['dt_date'],
        y=df['vl_dmp'],
        mode='lines',
        name='DI+',
        line=dict(color='green')  # Color for DI+
    )

    # Create the DI- trace
    minus_di_line = go.Scatter(
        x=df['dt_date'],
        y=df['vl_dmn'],
        mode='lines',
        name='DI-',
        line=dict(color='red')  # Color for DI-
    )
    # Create a fixed line for value 20 in light grey
    fixed_line = go.Scatter(
        x=df['dt_date'],
        y=[20] * len(df),  # Creating a list of 20s to match the length of the DataFrame
        mode='lines',
        name='Fixed Line (20)',
        line=dict(color='lightgrey', dash='dash')  # Color and style for the fixed line
    )

    # Create the figure for vl_adx chart
    fig = go.Figure(data=[vl_adx_line, plus_di_line, minus_di_line, fixed_line])

    # Update layout and add title
    fig.update_layout(
        title='vl_adx, DI+ and DI- Line Chart with Fixed Line at 20 using Plotly',
        xaxis_title='Date',
        yaxis_title='Values'
    )

    # Show the plot
    fig.show()

In [190]:
def ichimoku_chart(symbol): 
    df = df_concat.loc[df_concat['nm_symbol'] == f'{symbol}']
    # Create Ichimoku Cloud chart
    fig2 = go.Figure()

    # Add candlestick chart
    fig2.add_trace(go.Candlestick(x=df['dt_date'],
                                 open=df['vl_price_open'],
                                 high=df['vl_price_high'],
                                 low=df['vl_price_low'],
                                 close=df['vl_price_close'],
                                 name='Candlestick'))

    # Add Ichimoku Cloud lines
    fig2.add_trace(go.Scatter(x=df['dt_date'], y=df['conversion_line'], mode='lines', name='Conversion Line', line=dict(color='blue')))
    fig2.add_trace(go.Scatter(x=df['dt_date'], y=df['base_line'], mode='lines', name='Base Line', line=dict(color='red')))
    fig2.add_trace(go.Scatter(x=df['dt_date'], y=df['vl_leading_span_a'], mode='lines', name='Leading Span A'))
    fig2.add_trace(go.Scatter(x=df['dt_date'], y=df['leading_span_b'], mode='lines', name='Leading Span B'))

    # Update layout
    fig2.update_layout(title='Price Candlestick with Ichimoku Cloud Indicators',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      showlegend=True)

    # Display fig2ure
    fig2.show()

In [191]:
def combined_charts(symbol):
    vl_adx_chart(symbol)
    ichimoku_chart(symbol)

# Chamar a função para criar os gráficos lado a lado
combined_charts('ATOMUSDT')


KeyError: 'conversion_line'